In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [9]:
df = pd.read_csv('train.csv')

In [10]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# let's plan

In [11]:
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [12]:
X_train,X_test, Y_train, Y_test = train_test_split(df.drop(columns = ['Survived']),
                                                  df['Survived'],
                                                  test_size = 0.2,
                                                  random_state = 42)

In [13]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [14]:
Y_train.sample(5)

8      1
409    0
339    0
543    1
501    0
Name: Survived, dtype: int64

In [15]:
# imputation transformer
trfr1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy = 'most_frequent'), [6])
], remainder = 'passthrough')

In [16]:
# onehot encoding
trfr2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output = False, handle_unknown = 'ignore'), [1, 6])
], remainder = 'passthrough')

In [17]:
# scaling
trfr3 = ColumnTransformer([
    ('trfr3', MinMaxScaler(), slice(0, 10))
])

In [18]:
# feature scaling
trfr4 = SelectKBest(score_func = chi2, k = 8)

In [19]:
# train model
trfr5 = DecisionTreeClassifier()

# creating pipeline

In [20]:
pipe = Pipeline([
    ('trfr1', trfr1),
    ('trfr2', trfr2),
    ('trfr3', trfr3),
    ('trfr4', trfr4),
    ('trfr5', trfr5)
])

# Pipeline Vs Make_pipeline
#### pipeline requires naming of steps, make_pipeline doesn't.
#### (Same applies to ColumnTransformer vs make_transformer)

In [ ]:
# Alternate Syntax
pipe = make_pipeline(trfr1, trfr2, trfr3, trfr4, trfr5)

# which one should use if im using only 3 step pipeline should i use pipe.fit(X_train) or pipe,fit_transform(X_train) accoring to video fit_transform(X_train) should be use and according to chat gpt we can use pipe.fit(X_train) is sufficent and will give same result 

In [21]:
# train  
pipe.fit(X_train, Y_train)

Pipeline(steps=[('trfr1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trfr2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trfr3',
                 ColumnTransformer(transformers=[('trfr3', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trfr4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000243BC59D580>)),
                ('trfr5', DecisionTreeClassifier())])

In [22]:
# display pipeline
from sklearn import set_config
set_config(display = 'diagram')

In [23]:
# code here
pipe.named_steps['trfr1'].transformers_[0][1].statistics_

array([29.49884615])

In [24]:
pipe.named_steps['trfr1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [25]:
# predict
Y_pred = pipe.predict(X_test)

In [26]:
Y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.6256983240223464

# cross Validatiom using Pipeline

In [28]:
# cross vlidation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, Y_train, cv = 5, scoring = 'accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [39]:
params = {
    'trfr5__max_depth': [1, 2, 3, 4, 5, None]  # Correct format
}


In [40]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv = 5, scoring = 'accuracy')
grid.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trfr1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trfr2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trfr3',
                                        ColumnTransformer(transformers=[('trfr3',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trfr4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x00000243BC59D580>)),
                                       ('trfr5', DecisionTreeClassifier())]),
             param_grid={'trfr5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [41]:
grid.best_score_

0.6391214419383433

In [43]:
grid.best_params_

{'trfr5__max_depth': 2}

# Exporting the pipeline

In [44]:
# exporting
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))